In [1]:
import sys
sys.path.insert(1, '../../../..')

In [2]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.PINN import PINN
from methods.methodsDataset.PINNDataset import PINNDataset
from methods.DataDrivenMethods import DDMethod
from solvers.PoissonSolver import PoissonSolver

In [3]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': None, 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_PINN = {'layer_dims': None, 'activations': None, 'device': device, 'seed': 123}

In [4]:
def logn_fct(grid, num_fct, sigma=0.1, centers=None):
    D_logs = []

    if centers is None:
         centers = np.random.uniform(0.2, 0.8, num_fct)
        
    for c in centers:
        D_log = np.exp(-(grid - c)**2 / (2 * sigma**2)) + 1e-1
        D_logs.append(D_log)
    
    return np.stack(D_logs)

In [5]:
def delta(y, x, dy = 1., dx = 0.) :
    """    
    y : int, float or ndarray of size 1
    x : ndarray
    
    return dy if x = y and dx otherwise
    """
    if torch.is_tensor(y):
        return torch.where(torch.isclose(torch.Tensor(x), y), dy, dx)
    if not isinstance(y, np.ndarray):
        y = np.array(y)
    return np.where(np.isclose(np.array(x), y), dy, dx)

In [6]:
solver = PoissonSolver(params=params_solver)

In [7]:
Y_list = solver.x[20:81].reshape(-1, 1)
VD = logn_fct(solver.x, num_fct=61, centers=Y_list)
VF = delta(Y_list, solver.x)

In [8]:
# Vectorized solver
VU = solver.Vsolve(vect = 'DF', D=VD, F=VF)

In [9]:
print(VD.shape, VF.shape, VU.shape)

(61, 101) (61, 101) (61, 101)


In [10]:
VDF = np.concatenate([np.expand_dims(VD, -1), np.expand_dims(VF, -1)], axis=-1)

print(VDF.shape)

(61, 101, 2)


In [11]:
df_train, df_val, u_train, u_val = sklearn.model_selection.train_test_split(VDF, VU, test_size=0.2, random_state=123)

In [12]:
nx = params_solver['nx']
DF_train = torch.Tensor(df_train.reshape(df_train.shape[0]* df_train.shape[1], df_train.shape[2]))
DF_val = torch.Tensor(df_val.reshape(df_val.shape[0] * df_val.shape[1], df_val.shape[2]))

x = torch.Tensor(solver.x).view(-1, 1)
X_train = x.repeat(df_train.shape[0], 1)
X_val = x.repeat(df_val.shape[0], 1)

In [23]:
DF_train.shape

torch.Size([4848, 2])

In [13]:
DFX_train = torch.cat((DF_train, X_train), dim=1)
DFX_val = torch.cat((DF_val, X_val), dim=1)

U_train = torch.Tensor(u_train).view(-1, 1)
U_val = torch.Tensor(u_val).view(-1, 1)

In [14]:
def get_dataset():
    return U_train, U_val, DFX_train, DFX_val

In [15]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [16]:
def define_model(trial, input_size, output_size):
    n_layers = trial.suggest_int("n_layers", 2, 8)
    layers = [input_size]
    activations = []
    for i in range(n_layers):
        out_features = trial.suggest_int("units_l{}".format(i), 8, 120)
        activation = trial.suggest_categorical("activation_l{}".format(i), ["tanh", "relu"])
        layers += [out_features]
        activations.append(activation)
        
    layers += [output_size]
    params_PINN_trial = copy.deepcopy(params_PINN)
    params_PINN_trial['layer_dims'] = layers
    params_PINN_trial['activations'] = activations
    return PINN(params={'solver':params_solver, 'method':params_PINN_trial})

In [17]:
def objective(trial):
    # Generate the model.
    
    model = define_model(trial, 3, 1)
    # Generate the optimizers.
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    
    U_train, U_val, DFX_train, DFX_val = get_dataset()
    
    torch.manual_seed(params_PINN['seed'])
    
    n_epochs = 20000
    hyperparameters = {'lr': lr, 'epochs': n_epochs, 'optimizer': 'Adam'}
    # Training of the model.
    
    # w_r = trial.suggest_float("w_r", 1e-1, 1e+3, log=True)
    model.fit(hyperparameters = hyperparameters, 
              U_train=U_train, U_val=U_val, DX_train=DFX_train, DX_val=DFX_val,
              physics_ratio = 0.,
              data_ratio = 1.)
    
    V = model.loss_dict['val']
    # R = model.loss_dict['train']['residual']
    # B = model.loss_dict['train']['ic_bc']
    
    # S = [r + b for r, b in zip(R, B)]
    m = min(V)
    
    trial.report(m, n_epochs)

    return m

In [18]:
sampler = TPESampler(seed=params_PINN['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-18 15:23:35,059] A new study created in memory with name: no-name-d9af4310-5c79-4b48-aff4-7a6d92e58a46


In [19]:
study.optimize(objective, n_trials=10)

[tr : 2.6e-06, val : 2.7e-06]: 100%|████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:38<00:00, 521.70it/s]
[I 2023-08-18 15:24:14,128] Trial 0 finished with value: 2.5335914415336447e-06 and parameters: {'n_layers': 6, 'units_l0': 40, 'activation_l0': 'relu', 'units_l1': 89, 'activation_l1': 'relu', 'units_l2': 85, 'activation_l2': 'tanh', 'units_l3': 46, 'activation_l3': 'tanh', 'units_l4': 14, 'activation_l4': 'relu', 'units_l5': 28, 'activation_l5': 'relu', 'lr': 0.0003939877885425333}. Best is trial 0 with value: 2.5335914415336447e-06.
[tr : 2.7e-06, val : 3.0e-06]: 100%|████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:39<00:00, 512.77it/s]
[I 2023-08-18 15:24:53,138] Trial 1 finished with value: 2.4259891233668895e-06 and parameters: {'n_layers': 6, 'units_l0': 103, 'activation_l0': 'tanh', 'units_l1': 89, 'activation_l1': 'relu', 'units_l2': 33, 'activation_l2': 'rel

In [20]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  0
  Number of complete trials:  10
Best trial:
  Value:  2.354245680180611e-06
  Params: 
    n_layers: 6
    units_l0: 21
    activation_l0: relu
    units_l1: 105
    activation_l1: relu
    units_l2: 119
    activation_l2: relu
    units_l3: 21
    activation_l3: tanh
    units_l4: 69
    activation_l4: tanh
    units_l5: 55
    activation_l5: relu
    lr: 0.00033983414481272844


In [21]:
activations = []
layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'activation':
        activations.append(value)
    elif key.split('_')[0] == 'units':
        layer_dims.append(value)
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value
    elif key == 'w_r':
        w_r = value

Use this to fill config_step_1.py

In [22]:
print(activations, '\n', layer_dims, '\n', lr, '\n')

['relu', 'relu', 'relu', 'tanh', 'tanh', 'relu'] 
 [21, 105, 119, 21, 69, 55] 
 0.00033983414481272844 

